In [30]:
import pandas as pd
import warnings
warnings.simplefilter("ignore")

ta = pd.read_csv('data/traffic_accident.csv')

ta = ta.drop([0, 1], axis=0)
ta = ta.drop(['자치구별(1)', '2022.1', '2022.2', '2022.3', '2022.4', '2022.5'], axis=1)

ta.rename(columns={'자치구별(2)':'차치구', '2022':'발생건수'}, inplace=True)
ta.head()
print(ta.isnull().sum())

차치구     0
발생건수    0
dtype: int64


In [31]:
sb = pd.read_excel('data/speed_bump.xlsx')
print(sb.head())
print(sb.isnull().sum())

bump_counts = sb['시군구명'].value_counts().reset_index()
bump_counts.columns = ['시군구명', '과속방지턱수']

   번호 과속방지턱관리번호       도로명    시도명 시군구명               소재지도로명주소 소재지지번주소 설치장소  \
0   1       500       탑골로  서울특별시  금천구     서울특별시 금천구 탑골로 77-2     NaN   도로   
1   2       501       탑골로  서울특별시  금천구       서울특별시 금천구 탑골로 64     NaN   도로   
2   3       502     탑골로8길  서울특별시  금천구     서울특별시 금천구 탑골로8길 21     NaN   도로   
3   4       596  시흥대로147길  서울특별시  금천구  서울특별시 금천구 시흥대로147길 45     NaN   도로   
4   5       597  시흥대로148길  서울특별시  금천구  서울특별시 금천구 시흥대로148길 21     NaN   도로   

     과속방지턱재료 과속방지턱형태구분  ...  도로유형구분  규격여부    WGS84위도     WGS84경도 보차분리여부  \
0  아스팔트 콘크리트       원호형  ...      일반     Y  37.453350  126.917000      Y   
1  아스팔트 콘크리트       원호형  ...      일반     Y  37.452161  126.915942      Y   
2  아스팔트 콘크리트       원호형  ...      일반     Y  37.451083  126.914619      N   
3  아스팔트 콘크리트       원호형  ...      일반     Y  37.473679  126.895943      Y   
4  아스팔트 콘크리트       원호형  ...      일반     Y  37.475106  126.899968      Y   

   연속형여부  과속방지턱설치연도     관리기관명      관리기관전화번호     데이터기준일자  
0      N     2010.0  금천구청 도로

In [32]:
from scipy import stats
merged_data = pd.merge(bump_counts, ta, left_on='시군구명', right_on='차치구')
merged_data['발생건수'] = pd.to_numeric(merged_data['발생건수'], errors='coerce')

In [33]:
median_accidents = merged_data['발생건수'].median()
high_accident_areas = merged_data[merged_data['발생건수'] > median_accidents]['과속방지턱수']
low_accident_areas = merged_data[merged_data['발생건수'] <= median_accidents]['과속방지턱수']

In [34]:
_, p_high = stats.shapiro(high_accident_areas)
_, p_low = stats.shapiro(low_accident_areas)

if p_high > 0.05 and p_low > 0.05:
    print("두 그룹 모두 정규 분포를 따름")
else:
    print("그룹 중 하나 또는 둘 다 정규 분포를 따르지 않음.")

두 그룹 모두 정규 분포를 따름


In [35]:
_, p_levene = stats.levene(high_accident_areas, low_accident_areas)

if p_levene > 0.05:
    print("두 그룹의 분산이 동일함")
else:
    print("T두 그룹의 분산이 동일 하지 않음")


두 그룹의 분산이 동일함


In [41]:
t_stat, p_val = stats.ttest_ind(high_accident_areas, low_accident_areas, equal_var=(p_levene > 0.05))

print(f'T-statistic: {t_stat}')
print(f'P-value: {p_val}')

T-statistic: 0.7665568867071915
P-value: 0.4545073067016989
